## チャット返答

In [6]:
# 基本ライブラリ
!pip install python-dotenv
!pip install openai

# LangChain + FAISS 関連
!pip install langchain
!pip install langchain-community
!pip install faiss-cpu  # Mac/Windows/CPU環境用（GPUの場合は faiss-gpu）

# Hugging Face の埋め込みモデル使用のため
!pip install sentence-transformers

# PDF・Wordなどの読み込み用
!pip install unstructured
!pip install "unstructured[local-inference]"
!pip install "unstructured[pdf]"

python(63365) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


python(63366) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


python(63367) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


python(63372) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


python(63373) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import os
from dotenv import load_dotenv
import openai

from collect_law_texts import collect_law_texts_list
from extract_lifetime_azure import extract_lifetime_info_azure
from faiss_index_builder import build_faiss_index
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# === STEP 0: 初期設定 ===
load_dotenv()
user_chat = "パソコンとエアコンの法定耐用年数を教えて"

# === STEP 1: PDFをベクトル化し保存（初回以外は省略可） ===
# build_faiss_index(filename="ey-japan-info-sensor-2023-06-03.pdf")

# === STEP 2: FAISSインデックスを読み込み、類似情報取得 ===
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
index = FAISS.load_local(
    folder_path="storage",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
retrieved_docs = index.similarity_search(user_chat, k=2)
retrieved_context = "\n".join([doc.page_content for doc in retrieved_docs])

# === STEP 3: XML + txt から耐用年数情報を取得 ===
law_list = collect_law_texts_list("document")
input_text = user_chat
lifetime_info = extract_lifetime_info_azure(input_text, law_list)

# === STEP 4: 減価償却に関する法令テキストを読み込み ===
txt_file = "減価償却に関する法令.txt"
txt_path = os.path.join("document", txt_file)
txt_content = ""
if os.path.exists(txt_path):
    with open(txt_path, encoding="utf-8") as f:
        txt_content = f.read()
else:
    print(f"{txt_file} が見つかりませんでした")

# === STEP 5: Azure OpenAI に質問を投げる ===
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

client = openai.AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

prompt = f"""
あなたは日本の会計に精通した経理アシスタントAIです。
以下3つの会計に関する情報です。
--- 情報１：以下は有形固定資産に関連する会計基準等のまとめ ---
{retrieved_context}

--- 情報２：以下はユーザーからの情報をもとに抽出した法定耐用年数の情報です ---
{lifetime_info}

--- 情報３：以下は減価償却に関する法令です ---
{txt_content}

--- 以下はユーザーからの質問です ---
{user_chat}

【タスク】
- ３つの情報をもとにユーザーからの質問に回答する。各品目に対しては、適切な勘定科目を推定し、次の形式で出力してください：

【出力例】
品目名: ノートPC
勘定科目: 備品
法定耐用年数: ○○年

品目名: エアコン
勘定科目: 建物附属設備
法定耐用年数: ○○年

- 根拠があれば文書の抜粋も添えてください。
"""

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "あなたは日本の会計に精通した経理アシスタントAIです。"},
        {"role": "user", "content": prompt}
    ],
    temperature=0.2,
    max_tokens=2048
)

# === STEP 6: 出力表示 ===
print("\n\n==== 回答 ====")
print(response.choices[0].message.content)



==== 回答 ====
品目名: パソコン  
勘定科目: 器具及び備品  
法定耐用年数: 4年  

品目名: エアコン  
勘定科目: 器具及び備品  
法定耐用年数: 6年  

【根拠】  
情報２にて、パソコンは「器具及び備品」分類の「事務機器及び通信機器」に該当し、耐用年数は4年とされています。  
同じくエアコンは「器具及び備品」分類の「冷房用又は暖房用機器」に該当し、耐用年数は6年とされています。  

また、情報１の有形固定資産に関する会計基準では、同一条件の資産は同じ耐用年数を適用することが求められており、情報３の法令に基づく法定耐用年数の適用が一般的であるため、上記の法定耐用年数を適用するのが妥当です。


### 以下個別実行

In [7]:
import os
from dotenv import load_dotenv
import openai

from collect_law_texts import collect_law_texts_list
from extract_lifetime_azure import extract_lifetime_info_azure
from faiss_index_builder import build_faiss_index
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

In [8]:
import pandas as pd
example_dir = "document/example_accounting_entry/"
os.makedirs(example_dir, exist_ok=True)

# エクセル読み込み
excel_files = [f for f in os.listdir(example_dir) if f.endswith(('.xlsx', '.xls'))]

In [9]:
excel_files

['固定資産台帳_サンプル.xlsx']

In [10]:
dataframes = {}
for file in excel_files:
    file_path = os.path.join(example_dir, file)
    try:
        df = pd.read_excel(file_path)
        dataframes[file] = df
        print(f"\n {file} の内容:")
        print(df.head())
    except Exception as e:
        print(f"{file} の読み込み中にエラーが発生しました: {e}")


 固定資産台帳_サンプル.xlsx の内容:
     資産番号                 資産名    資産分類     管理部門      設置場所   メーカー        型番  \
0  FA-001  ノートPC（MacBook Pro）    器具備品  情報システム部      本社3F  Apple     A2485   
1  FA-002                空調設備  建物附属設備      総務部      東京支店   ダイキン   DAI-800   
2  FA-003           乗用車（プリウス）   車両運搬具      営業部      関東支社    トヨタ     ZVW50   
3  FA-004       複合機（コピー/スキャナ）    器具備品      総務部      本社2F    リコー  IM C3000   
4  FA-005                倉庫建物      建物      管理部  千葉物流センター  竹中工務店         -   

          取得日  取得価額 (円) 耐用年数 償却方法 資産状態      備考  
0  2022/04/01    250000   4年  定額法  使用中    技術者用  
1  2020/08/15    800000  13年  定額法  使用中   年1回点検  
2  2023/06/01   2300000   6年  定額法  使用中     役員車  
3  2021/11/20    500000   5年  定額法  使用中  保守契約あり  
4  2015/03/10  10000000  38年  定額法  使用中   耐震補強済  


In [11]:
dataframes[file] 

,資産番号,資産名,資産分類,管理部門,設置場所,メーカー,型番,取得日,取得価額 (円),耐用年数,償却方法,資産状態,備考
0,FA-001,ノートPC（MacBook Pro）,器具備品,情報システム部,本社3F,Apple,A2485,2022/04/01,250000,4年,定額法,使用中,技術者用
1,FA-002,空調設備,建物附属設備,総務部,東京支店,ダイキン,DAI-800,2020/08/15,800000,13年,定額法,使用中,年1回点検
2,FA-003,乗用車（プリウス）,車両運搬具,営業部,関東支社,トヨタ,ZVW50,2023/06/01,2300000,6年,定額法,使用中,役員車
3,FA-004,複合機（コピー/スキャナ）,器具備品,総務部,本社2F,リコー,IM C3000,2021/11/20,500000,5年,定額法,使用中,保守契約あり
4,FA-005,倉庫建物,建物,管理部,千葉物流センター,竹中工務店,-,2015/03/10,10000000,38年,定額法,使用中,耐震補強済
5,FA-006,生産ライン制御装置,機械装置,製造部,工場A棟,オムロン,PLC-X300,2019/05/01,3200000,10年,定率法,使用中,自動ライン制御用
6,FA-007,組立用ロボットアーム,機械装置,製造部,工場A棟,ファナック,R-30iB,2020/06/15,5500000,11年,定額法,使用中,1号ライン専用
7,FA-008,CADソフトウェアライセンス,無形固定資産,設計部,本社5F,Autodesk,AutoCAD 2022,2022/10/01,600000,5年,定額法,使用中,年更新あり
8,FA-009,デジタルサイネージ,器具備品,販売促進部,本社1F,Panasonic,DS-65,2021/07/10,420000,5年,定額法,使用中,店頭広告用
9,FA-010,リーチ式フォークリフト,車両運搬具,ロジ部,倉庫B棟,ニチユ,FBR15,2018/09/25,1800000,6年,定率法,使用中,リーチタイプ


In [13]:
# 複数Excelファイルの仕訳例を文字列化してまとめる
accounting_examples_text = ""

for file, df in dataframes.items():
    accounting_examples_text += f"\n■ {file} の仕訳例:\n"
    accounting_examples_text += df.to_string(index=False)
    accounting_examples_text += "\n"

In [14]:
accounting_examples_text

'\n■ 固定資産台帳_サンプル.xlsx の仕訳例:\n  資産番号                資産名   資産分類    管理部門      設置場所      メーカー             型番        取得日  取得価額 (円) 耐用年数 償却方法 資産状態       備考\nFA-001 ノートPC（MacBook Pro）   器具備品 情報システム部      本社3F     Apple          A2485 2022/04/01    250000   4年  定額法  使用中     技術者用\nFA-002               空調設備 建物附属設備     総務部      東京支店      ダイキン        DAI-800 2020/08/15    800000  13年  定額法  使用中    年1回点検\nFA-003          乗用車（プリウス）  車両運搬具     営業部      関東支社       トヨタ          ZVW50 2023/06/01   2300000   6年  定額法  使用中      役員車\nFA-004      複合機（コピー/スキャナ）   器具備品     総務部      本社2F       リコー       IM C3000 2021/11/20    500000   5年  定額法  使用中   保守契約あり\nFA-005               倉庫建物     建物     管理部  千葉物流センター     竹中工務店              - 2015/03/10  10000000  38年  定額法  使用中    耐震補強済\nFA-006          生産ライン制御装置   機械装置     製造部      工場A棟      オムロン       PLC-X300 2019/05/01   3200000  10年  定率法  使用中 自動ライン制御用\nFA-007         組立用ロボットアーム   機械装置     製造部      工場A棟     ファナック         R-30iB 2020/06/15   5500000  11年  定額法  使用中  1号ラ

In [15]:
# STEP0
# PDFをベクトル化してFAISSに保存（毎回実行する必要あるか）
build_faiss_index(filename="ey-japan-info-sensor-2023-06-03.pdf")

The PDF <_io.BufferedReader name='/Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/document/ey-japan-info-sensor-2023-06-03.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


ファイル読み込み: /Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/document/ey-japan-info-sensor-2023-06-03.pdf
ドキュメント数: 1
チャンク数: 10
埋め込みモデル読み込み: intfloat/multilingual-e5-large
FAISSインデックス作成完了
インデックス保存済み: /Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/storage


In [25]:
user_chat = "パソコンとエアコンの法定耐用年数を教えて"

In [26]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# 1. ベクトル化で使ったモデルを再読み込み
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

# 2. FAISSインデックスを読み込み
index = FAISS.load_local(
    folder_path="storage",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
# 類似チャンクを抽出
retrieved_docs = index.similarity_search(user_chat, k=2)

In [33]:
#文字のみ
retrieved_context = "\n".join([doc.page_content for doc in retrieved_docs])

In [28]:
law_list = collect_law_texts_list("document")
input_text = user_chat


In [29]:
# ユーザーから送られたチャットもしくは画像から品目を抽出し法定耐用年数を出す
lifetime_info = extract_lifetime_info_azure(input_text, law_list)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
lifetime_info

'品目名: パソコン\n分類: 器具及び備品\n細目: ２\u3000事務機器及び通信機器 - パーソナルコンピュータ（サーバー用のものを除く。）\n耐用年数: 4年\n\n品目名: エアコン\n分類: 器具及び備品\n細目: 冷房用又は暖房用機器\n耐用年数: 6年'

In [31]:
txt_file = "減価償却に関する法令.txt"
txt_path = os.path.join("document", txt_file)
if os.path.exists(txt_path):
    with open(txt_path, encoding="utf-8") as f:
        txt_content = f.read()
else:
    print(f"{txt_file}' が見つかりませんでした")

In [32]:
txt_content

'（一般の減価償却資産の耐用年数）\n第一条\u3000所得税法（昭和四十年法律第三十三号）第二条第一項第十九号（定義）又は法人税法（昭和四十年法律第三十四号）第二条第二十三号（定義）に規定する減価償却資産（以下「減価償却資産」という。）のうち鉱業権（租鉱権及び採石権その他土石を採掘し又は採取する権利（二酸化炭素の貯留事業に関する法律（令和六年法律第三十八号）第二条第八項（定義）に規定する試掘権を除く。次項第三号において同じ。）を含む。以下同じ。）、二酸化炭素の貯留事業に関する法律第二条第八項に規定する試掘権、坑道、公共施設等運営権、樹木採取権及び漁港水面施設運営権以外のものの耐用年数は、次の各号に掲げる資産の区分に応じ当該各号に定める表に定めるところによる。\n一\u3000所得税法施行令（昭和四十年政令第九十六号）第六条第一号、第二号及び第四号から第七号まで（減価償却資産の範囲）又は法人税法施行令（昭和四十年政令第九十七号）第十三条第一号、第二号及び第四号から第七号まで（減価償却資産の範囲）に掲げる資産（坑道を除く。）\u3000別表第一（機械及び装置以外の有形減価償却資産の耐用年数表）\n二\u3000所得税法施行令第六条第三号又は法人税法施行令第十三条第三号に掲げる資産\u3000別表第二（機械及び装置の耐用年数表）\n三\u3000所得税法施行令第六条第八号又は法人税法施行令第十三条第八号に掲げる資産（鉱業権、二酸化炭素の貯留事業に関する法律第二条第八項に規定する試掘権、公共施設等運営権、樹木採取権及び漁港水面施設運営権を除く。）\u3000別表第三（無形減価償却資産の耐用年数表）\n四\u3000所得税法施行令第六条第九号又は法人税法施行令第十三条第九号に掲げる資産\u3000別表第四（生物の耐用年数表）\n２\u3000鉱業権、二酸化炭素の貯留事業に関する法律第二条第八項に規定する試掘権、坑道、公共施設等運営権、樹木採取権及び漁港水面施設運営権の耐用年数は、次の各号に掲げる資産の区分に応じ当該各号に定める年数とする。\n一\u3000採掘権\u3000当該採掘権に係る鉱区の採掘予定数量を、当該鉱区の最近における年間採掘数量その他当該鉱区に属する設備の採掘能力、当該鉱区において採掘に従事する人員の数等に照らし適正に推計される年間採掘数量で除して計算した数

In [11]:
# 内容を確認
print(f"law_list の件数: {len(law_list)}")
print("1件目のファイル名:", law_list[0]["ファイル名"])
print("1件目の内容（先頭300文字）:")
print(law_list[0]["内容"][:300])


law_list の件数: 14
1件目のファイル名: 機械及び装置以外の有形減価償却資産の耐用年数表（建物）.xml
1件目の内容（先頭300文字）:
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<Law Lang="ja" Era="Showa" Year="40" Num="015" PromulgateMonth="03" PromulgateDay="31" LawType="MinisterialOrdinance"><LawNum>昭和四十年大蔵省令第十五号</LawNum><LawBody><LawTitle Kana="げんかしょうきゃくしさんのたいようねんすうとうにかんするしょうれい" Abbrev="耐用年数省令" AbbrevKana="た">減価償却資産


In [21]:
print(f"🔍 類似チャンク件数: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n--- 類似チャンク {i} ---")
    print(doc.page_content[:500], "...")

🔍 類似チャンク件数: 2

--- 類似チャンク 1 ---
化の危険の程度、そのほか特殊的条件を考慮して自主

的に決定すべきとされています。

また、同一条件（種類・材質・構造・用途・環境等

が同一であること）の資産については、異なる耐用年

数の適用は認められないとされています。

次に、残存価額は、有形固定資産の耐用年数到来時

において予想される当該資産の売却価格又は利用価格

から解体、撤去、処分等の費用を控除した金額であり、

10　　情報センサー Vol.188 June 2023

の

耐用年数と同様に各企業が当該資産の特殊的条件を考

慮して合理的に見積りを行うべきものとされています。

しかし、耐用年数、残存価額について、多くの企業

が法人税法の規定に従っているのが現状であり、企業

の状況に照らし、不合理と認められる事情のない限

り、当面妥当なものとして取り扱うことができるとさ れています（減価償却取扱い第24項）。

6. 除却・売却

企業会計原則第三 五Dでは、耐用年数到来後の有形

固定資産は、除却されるまで残存価額又は備忘価額で

記載するとされています。

耐用年数到来後前に除却・売却した場合、その未償 ...

--- 類似チャンク 2 ---
的、規則的に実施しなければならないとされています （連続意見書第三 第一 二）。

前記の減価償却方法については、企業会計原則第三

五及び同注解（注20）で定められています。

ここで、企業会計原則及び同注解では、取得した有

形固定資産は、その取得原価を当該資産の耐用期間に

わたり、定額法、定率法等の一定の減価償却の方法に

よって、その取得原価を各事業年度に配分しなければ

ならないとされています。企業会計原則では、減価償

却の方法として、定額法、定率法、級数法及び生産高

比例法が掲げられており、実務上は法人税法の規定に

従った方法を採用している企業が多いと考えられます （参考：監査・保証実務委員会実務指針第81号「減価

償却に関する当面の監査上の取扱い」（以下、減価償

却取扱い））。この他、減価償却の方法ではありません

が、取替資産に係る費用配分の方法として取替法が挙

げられています。

また、この減価償却の方法は、企業が決定する会

計方針であるとされ

In [34]:
import openai
import os
from dotenv import load_dotenv

# .env 読み込み
load_dotenv()

# Azure OpenAI のクライアント初期化
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

client = openai.AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

In [35]:
prompt = f"""
あなたは日本の会計に精通した経理アシスタントAIです。
以下3つの会計に関する情報です。
--- 情報１：以下は有形固定資産に関連する会計基準等のまとめ ---
{retrieved_context}

--- 情報２：以下はユーザーからの情報をもとに抽出した法定耐用年数の情報です ---
{lifetime_info}

--- 情報３：以下は減価償却に関する法令です ---
{txt_content}

--- 以下はユーザーからの質問です ---
{user_chat}

【タスク】
- ３つの情報をもとにユーザーからの質問に回答する。各品目に対しては、適切な勘定科目を推定し、次の形式で出力してください：

【出力例】
品目名: ノートPC
勘定科目: 備品
法定耐用年数: ○○年

品目名: エアコン
勘定科目: 建物附属設備
法定耐用年数: ○○年

- 根拠があれば文書の抜粋も添えてください。
"""

In [18]:
# .envから環境変数を読み込む
load_dotenv()

True

In [36]:
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")
if not (api_key and azure_endpoint and deployment):
    raise ValueError("AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_DEPLOYMENT を環境変数で指定してください。")

client = openai.AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

In [37]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "あなたは日本の会計に精通した経理アシスタントAIです。"},
        {"role": "user", "content": prompt}
    ],
    temperature=0.2,
    max_tokens=2048
)

In [38]:
response

ChatCompletion(id='chatcmpl-Bandihe1JF4x9aL4GneCPEOn0FcuG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='品目名: パソコン  \n勘定科目: 器具及び備品  \n法定耐用年数: 4年  \n\n品目名: エアコン  \n勘定科目: 器具及び備品  \n法定耐用年数: 6年  \n\n【根拠】  \n情報２にて、パソコンは「器具及び備品」分類の「事務機器及び通信機器」に該当し、耐用年数は4年とされています。  \n同じくエアコンは「器具及び備品」分類の「冷房用又は暖房用機器」に該当し、耐用年数は6年とされています。  \n\nまた、情報１の有形固定資産に関する会計基準では、同一条件の資産には同一の耐用年数を適用することが求められており、情報３の法令に基づく耐用年数の適用もこれに準じています。', refusal=None, role='assistant', function_call=None, tool_calls=None, annotations=[]), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1748110322, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_178c8d546f', usage=CompletionUsage(completion_tokens=235, prompt_tokens=8899, total_tokens=9134, c

In [14]:
# api_key
# azure_endpoint
# deployment
# api_version

In [20]:
law_list = collect_law_texts_list("document")
print(law_list[0])  # 1件目を表示して確認

{'ファイル名': '機械及び装置以外の有形減価償却資産の耐用年数表（建物）.xml', '内容': '<?xml version="1.0" encoding="UTF-8" standalone="no"?>\n<Law Lang="ja" Era="Showa" Year="40" Num="015" PromulgateMonth="03" PromulgateDay="31" LawType="MinisterialOrdinance"><LawNum>昭和四十年大蔵省令第十五号</LawNum><LawBody><LawTitle Kana="げんかしょうきゃくしさんのたいようねんすうとうにかんするしょうれい" Abbrev="耐用年数省令" AbbrevKana="た">減価償却資産の耐用年数等に関する省令</LawTitle>\n    <AppdxTable>\n      <AppdxTableTitle >別表第一</AppdxTableTitle>\n      <RelatedArticleNum>\u3000機械及び装置以外の有形減価償却資産の耐用年数表</RelatedArticleNum>\n      <TableStruct>\n        <Table >\n          <TableRow>\n            <TableColumn >\n              <Sentence Num="1" >種類</Sentence>\n            </TableColumn>\n            <TableColumn >\n              <Sentence Num="1" >構造又は用途</Sentence>\n            </TableColumn>\n            <TableColumn >\n              <Sentence Num="1" >細目</Sentence>\n            </TableColumn>\n            <TableColumn >\n              <Sentence Num="1" >耐用年数</Sentence>\n            </TableColumn

In [21]:
# すべてのxmlファイルの内容を1つのテキストにまとめる
law_texts = ""
for item in law_list:
    law_texts += f"【{item['ファイル名']}】\n{item['内容']}\n\n"

In [27]:
input_text = "A車両本体 - ： 2,690,182円（税抜）"

In [30]:
retrieved_docs = index.similarity_search(input_text, k=2)
# テキストのみ抽出
retrieved_context = "\n".join([doc.page_content for doc in retrieved_docs])


In [32]:
retrieved_context

'土地など使用や時の経過を通じて価値が減少しないものをいい、減損処理を除いては、基本的 に費用配分は行われません。\n\n土地、美術品＊\n\n減耗性資産\n\n採掘あるいは採取されるにつれて漸次減耗し枯渇する天然資源をいい、全体としての用役を もって生産に役立つものではなく、採取されるに応じてその実態が部分的に製品化されるもの です（連続意見書第三 第一 六2参照）。\n\n鉱山や油田における埋蔵 資源、山林\n\n＊\u3000なお、税務上は、1点100万円未満等の一定の美術品は償却資産として取り扱われる。 出典：新日本有限責任監査法人編『固定資産の会計実務ハンドブック』（中央経済社、2011年）P.7の図表1-1-2を基に筆者一部修正\n\n▶表3\u3000形態別の取得原価\n\n形態\n\n取得原価\n\n購入\n\n有形固定資産を購入によって取得した場合には、購入代金に買入手数料、運送費、荷役費、据付費、試運転費等の付随費用を 加えて取得原価とする。ただし、正当な理由がある場合には、付随費用の一部又は全部を加算しない額をもって取得原価とす ることができる。 購入に際しては値引又は割戻を受けたときには、これを購入代金から控除する。\n\n自家建設\n\n固定資産を自家建設した場合には、適正な原価計算基準に従って製造原価を計算し、これに基づいて取得原価を計算する。建 設に要する借入資本の利子で稼働前の期間に属するものは、これを取得原価に算入することができる。\n\n現物出資\n\n株式を発行しその対価として固定資産を受け入れた場合には、出資者に対して交付された株式の発行価額をもって取得原価と する＊1。\n\n交換\n\n自己所有の固定資産と交換に固定資産を取得した場合には、交換に供された自己資産の適正な簿価をもって取得原価とする。 自己所有の株式ないし社債等と固定資産を交換した場合には、当該有価証券の時価又は適正な簿価をもって取得原価とする＊2。\n\n贈与\n\n固定資産を贈与された場合には、時価等を基準として公正に評価した額をもって取得原価とする。\n交換\n\n自己所有の固定資産と交換に固定資産を取得した場合には、交換に供された自己資産の適正な簿価をもって取得原価とする。 自己所有の株式ないし社債等と固定資産を交換した場合には、当該有価証券の時価又は

In [ ]:
prompt = f"""
あなたは日本の減価償却資産の法定耐用年数に詳しいAIです。
以下は法定耐用年数に関する法令xmlファイルの全文です。

{law_texts}

--- ここからINPUT_TEXT ---
{input_text}
--- ここまでINPUT_TEXT ---

【タスク】
- INPUT_TEXTに記載された各品目について、最も該当する資産分類・細目・耐用年数をxml群から探し、以下の形式で出力してください。

【出力例】
品目名: ○○○
分類: ○○○
細目: ○○○
耐用年数: ○○年

- 必ず全品目について出力してください。
- xmlに該当がなければ「該当なし」と記載してください。
"""

In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "あなたは減価償却資産の法定耐用年数に詳しいAIです。"},
        {"role": "user", "content": prompt}
    ],
    temperature=0.2,
    max_tokens=2048
)

## ▼ 7. 検索テスト

In [3]:
query = "固定資産とは何ですか？"
results = index.similarity_search(query, k=2)

for i, doc in enumerate(results, 1):
    print(f"\n--- 類似チャンク {i} ---")
    print(doc.page_content[:200])
    print(f"メタデータ: {doc.metadata}")


--- 類似チャンク 1 ---
土地など使用や時の経過を通じて価値が減少しないものをいい、減損処理を除いては、基本的 に費用配分は行われません。

土地、美術品＊

減耗性資産

採掘あるいは採取されるにつれて漸次減耗し枯渇する天然資源をいい、全体としての用役を もって生産に役立つものではなく、採取されるに応じてその実態が部分的に製品化されるもの です（連続意見書第三 第一 六2参照）。

鉱山や油田における埋蔵 資源、山林


メタデータ: {'source': '/Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/document/ey-japan-info-sensor-2023-06-03.pdf'}

--- 類似チャンク 2 ---
交換

自己所有の固定資産と交換に固定資産を取得した場合には、交換に供された自己資産の適正な簿価をもって取得原価とする。 自己所有の株式ないし社債等と固定資産を交換した場合には、当該有価証券の時価又は適正な簿価をもって取得原価とする＊2。

贈与

固定資産を贈与された場合には、時価等を基準として公正に評価した額をもって取得原価とする。

＊1　なお、企業会計基準第8号「ストック・オプション等に関
メタデータ: {'source': '/Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/document/ey-japan-info-sensor-2023-06-03.pdf'}
